In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [2]:
df = pd.read_csv('/Users/Ajwad/Desktop/NBM_Regression_Gamma/yellow_tripdata_2021-03.csv') # Load tha dataset
df

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2021-03-01 00:22:02,2021-03-01 00:23:22,1.0,0.00,1.0,N,264,264,2.0,3.00,0.50,0.5,0.00,0.00,0.3,4.30,0.0
1,2.0,2021-03-01 00:24:48,2021-03-01 00:24:56,1.0,0.00,1.0,N,152,152,2.0,2.50,0.50,0.5,0.00,0.00,0.3,3.80,0.0
2,2.0,2021-03-01 00:25:17,2021-03-01 00:31:01,1.0,0.00,1.0,N,152,152,2.0,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0
3,1.0,2021-03-01 00:07:40,2021-03-01 00:31:23,0.0,16.50,4.0,N,138,265,1.0,51.00,0.50,0.5,11.65,6.12,0.3,70.07,0.0
4,2.0,2021-03-01 00:02:13,2021-03-01 00:06:01,1.0,1.13,1.0,N,68,264,1.0,5.50,0.50,0.5,1.86,0.00,0.3,11.16,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925147,NaN,2021-03-29 12:39:29,2021-03-29 13:06:30,NaN,13.18,NaN,NaN,135,75,NaN,34.76,2.75,0.5,0.00,6.12,0.3,44.43,0.0
1925148,NaN,2021-03-29 12:11:00,2021-03-29 12:28:00,NaN,2.60,NaN,NaN,107,163,NaN,18.34,2.75,0.5,0.00,0.00,0.3,21.89,0.0
1925149,NaN,2021-03-29 12:50:00,2021-03-29 13:14:00,NaN,5.50,NaN,NaN,137,17,NaN,37.76,2.75,0.5,0.00,0.00,0.3,41.31,0.0
1925150,NaN,2021-03-29 12:12:00,2021-03-29 12:32:00,NaN,3.78,NaN,NaN,177,85,NaN,-1.55,2.75,0.5,0.00,23.50,0.3,25.50,0.0


In [3]:
df.shape

(1925152, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925152 entries, 0 to 1925151
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 264.4+ MB


In [5]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [6]:
#-----------------------------

In [7]:
df_train , df_test = train_test_split(df , test_size=.2, random_state=10)

In [8]:
df_train, df_val = train_test_split(df_train, test_size=.25, random_state=5)

In [9]:
#------- clean train 

In [10]:
df_train.shape

(1155090, 18)

In [11]:
nulls = df_train.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,76724
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,76724
trip_distance,0
RatecodeID,76724
store_and_fwd_flag,76724
PULocationID,0
DOLocationID,0
payment_type,76724


In [12]:
df_train = df_train.dropna(how ='any')

In [13]:
nulls = df_train.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
store_and_fwd_flag,0
PULocationID,0
DOLocationID,0
payment_type,0


In [14]:
df_train['Day'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.day

In [15]:
df_train['Hour_pickup'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.hour
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
964410,2.0,2021-03-17 23:23:57,2021-03-17 23:28:16,1.0,1.92,1.0,N,170,141,1.0,7.0,0.5,0.5,1.50,0.0,0.3,12.30,2.5,17,23
1602218,1.0,2021-03-28 15:28:23,2021-03-28 15:33:27,3.0,1.10,1.0,N,236,239,1.0,6.0,2.5,0.5,1.00,0.0,0.3,10.30,2.5,28,15
1502881,2.0,2021-03-26 19:42:37,2021-03-26 20:01:13,1.0,2.19,1.0,N,87,148,2.0,13.5,1.0,0.5,0.00,0.0,0.3,17.80,2.5,26,19
570508,2.0,2021-03-11 12:16:29,2021-03-11 12:31:32,1.0,2.02,1.0,N,186,113,1.0,11.0,0.0,0.5,2.86,0.0,0.3,17.16,2.5,11,12
1374167,2.0,2021-03-24 19:03:46,2021-03-24 19:13:03,1.0,2.38,1.0,N,50,238,1.0,9.5,1.0,0.5,2.76,0.0,0.3,16.56,2.5,24,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245787,2.0,2021-03-22 18:15:08,2021-03-22 18:22:35,1.0,1.59,1.0,N,186,113,1.0,7.5,1.0,0.5,1.00,0.0,0.3,12.80,2.5,22,18
1409311,1.0,2021-03-25 13:33:35,2021-03-25 13:52:50,1.0,6.70,1.0,N,141,231,1.0,22.5,2.5,0.5,5.15,0.0,0.3,30.95,2.5,25,13
1092821,2.0,2021-03-19 20:19:08,2021-03-19 20:26:36,1.0,1.27,1.0,N,186,68,1.0,7.0,0.5,0.5,2.70,0.0,0.3,13.50,2.5,19,20
1629764,1.0,2021-03-29 10:18:41,2021-03-29 10:30:52,1.0,2.60,1.0,N,226,229,2.0,11.5,2.5,0.5,0.00,0.0,0.3,14.80,2.5,29,10


In [16]:
df1_train= df_train.drop(df_train[df_train.Day < 20 ].index)
df1_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
1602218,1.0,2021-03-28 15:28:23,2021-03-28 15:33:27,3.0,1.10,1.0,N,236,239,1.0,6.0,2.5,0.5,1.00,0.0,0.3,10.30,2.5,28,15
1502881,2.0,2021-03-26 19:42:37,2021-03-26 20:01:13,1.0,2.19,1.0,N,87,148,2.0,13.5,1.0,0.5,0.00,0.0,0.3,17.80,2.5,26,19
1374167,2.0,2021-03-24 19:03:46,2021-03-24 19:13:03,1.0,2.38,1.0,N,50,238,1.0,9.5,1.0,0.5,2.76,0.0,0.3,16.56,2.5,24,19
1398315,2.0,2021-03-25 11:28:54,2021-03-25 11:32:21,1.0,0.86,1.0,N,236,236,1.0,5.0,0.0,0.5,1.66,0.0,0.3,9.96,2.5,25,11
1201170,2.0,2021-03-21 22:19:23,2021-03-21 22:31:40,2.0,2.88,1.0,N,166,239,1.0,12.0,0.5,0.5,3.95,0.0,0.3,19.75,2.5,21,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118566,2.0,2021-03-20 12:25:12,2021-03-20 12:36:04,1.0,1.83,1.0,N,186,237,1.0,9.0,0.0,0.5,2.46,0.0,0.3,14.76,2.5,20,12
1455407,2.0,2021-03-26 09:19:16,2021-03-26 09:28:55,2.0,1.54,1.0,N,186,161,1.0,8.5,0.0,0.5,2.95,0.0,0.3,14.75,2.5,26,9
1245787,2.0,2021-03-22 18:15:08,2021-03-22 18:22:35,1.0,1.59,1.0,N,186,113,1.0,7.5,1.0,0.5,1.00,0.0,0.3,12.80,2.5,22,18
1409311,1.0,2021-03-25 13:33:35,2021-03-25 13:52:50,1.0,6.70,1.0,N,141,231,1.0,22.5,2.5,0.5,5.15,0.0,0.3,30.95,2.5,25,13


In [17]:
df1_train['Day'] = pd.to_datetime(df_train['tpep_pickup_datetime']).dt.weekday

In [18]:
df1_train.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','tpep_pickup_datetime','tpep_dropoff_datetime']
         ,axis=1, inplace=True ) # delete two columns ,(axis=1:refers to column dimension)

In [64]:
duplicate = df1_train.duplicated()
print(duplicate.sum())
df1_train[duplicate]

37956


,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
1270134,2.0,1.0,3.36,2.0,1.0,14.5,0.0,0.5,3.56,0.0,0.3,21.36,2.5,1,10
1192849,1.0,1.0,0.80,1.0,1.0,4.5,2.5,0.5,1.55,0.0,0.3,9.35,2.5,6,18
1386492,1.0,1.0,1.10,1.0,2.0,7.5,2.5,0.5,0.00,0.0,0.3,10.80,2.5,3,8
1641486,1.0,1.0,0.80,1.0,1.0,6.0,2.5,0.5,1.85,0.0,0.3,11.15,2.5,0,13
1756520,1.0,1.0,1.60,1.0,2.0,8.0,2.5,0.5,0.00,0.0,0.3,11.30,2.5,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464853,1.0,1.0,1.20,1.0,2.0,7.0,2.5,0.5,0.00,0.0,0.3,10.30,2.5,4,11
1377045,2.0,1.0,0.71,2.0,1.0,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,2,20
1750936,1.0,1.0,2.10,1.0,1.0,9.5,2.5,0.5,2.55,0.0,0.3,15.35,2.5,2,11
1383289,1.0,1.0,0.60,1.0,2.0,4.5,2.5,0.5,0.00,0.0,0.3,7.80,2.5,3,6


In [65]:
df1_train.drop_duplicates(inplace=True)

In [66]:
duplicate = df1_train.duplicated()
print(duplicate.sum())
df1_train[duplicate]

0


,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup


In [67]:
df1_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378935 entries, 1602218 to 1409311
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               378935 non-null  float64
 1   passenger_count        378935 non-null  float64
 2   trip_distance          378935 non-null  float64
 3   RatecodeID             378935 non-null  object 
 4   payment_type           378935 non-null  float64
 5   fare_amount            378935 non-null  float64
 6   extra                  378935 non-null  float64
 7   mta_tax                378935 non-null  float64
 8   tip_amount             378935 non-null  float64
 9   tolls_amount           378935 non-null  float64
 10  improvement_surcharge  378935 non-null  float64
 11  total_amount           378935 non-null  float64
 12  congestion_surcharge   378935 non-null  float64
 13  Day                    378935 non-null  int64  
 14  Hour_pickup            378935

In [68]:
Day_hours = df1_train.groupby('Hour_pickup').Day.value_counts().sort_values(ascending=False)
Day_hours

Hour_pickup  Day
17           2      5508
18           2      5504
15           2      5213
18           5      5079
17           5      5049
                    ... 
4            4        59
3            3        54
             4        52
2            3        48
4            3        48
Name: Day, Length: 168, dtype: int64

In [69]:
df1_train.corr()

,VendorID,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
VendorID,1.000000,0.169116,0.001287,-0.013850,-0.022084,-0.855360,-0.032113,0.006165,-0.009040,-0.048113,-0.019021,0.008342,-0.007882,0.001353
passenger_count,0.169116,1.000000,0.004987,-0.001275,-0.000250,-0.139288,0.011971,-0.000595,0.002644,0.008981,0.001349,0.016819,0.018841,0.024197
trip_distance,0.001287,0.004987,1.000000,-0.002175,0.876596,-0.002464,-0.006673,0.394868,0.491562,0.020994,0.864144,-0.177717,0.020727,0.017701
payment_type,-0.013850,-0.001275,-0.002175,1.000000,-0.046602,-0.038588,-0.290951,-0.478458,-0.017633,-0.318393,-0.149676,-0.258499,-0.009299,-0.034570
fare_amount,-0.022084,-0.000250,0.876596,-0.046602,1.000000,0.001221,0.052994,0.416823,0.495707,0.148833,0.976689,-0.157118,0.015490,0.014824
extra,-0.855360,-0.139288,-0.002464,-0.038588,0.001221,1.000000,0.087616,0.031821,-0.001805,0.082622,0.032804,0.195095,-0.063604,0.155455
mta_tax,-0.032113,0.011971,-0.006673,-0.290951,0.052994,0.087616,1.000000,0.022469,-0.056210,0.908681,0.083382,0.430405,-0.007489,0.000252
tip_amount,0.006165,-0.000595,0.394868,-0.478458,0.416823,0.031821,0.022469,1.000000,0.312230,0.065046,0.576040,0.037314,0.014750,0.040900
tolls_amount,-0.009040,0.002644,0.491562,-0.017633,0.495707,-0.001805,-0.056210,0.312230,1.000000,0.022254,0.572416,-0.126138,0.008931,0.004347
improvement_surcharge,-0.048113,0.008981,0.020994,-0.318393,0.148833,0.082622,0.908681,0.065046,0.022254,1.000000,0.179504,0.411917,-0.002540,-0.002099


In [ ]:
sns.pairplot(df1_train);

In [ ]:
plt.figure(figsize = [15,15])
sns.heatmap(df1_train.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
sns.boxplot(data=df1_train);
plt.xticks(rotation = 90)
plt.title("Detecting outliers using Boxplot")

In [22]:
#------- End clean train 

In [23]:
#------ Clean Val

In [24]:
df_val.shape

(385031, 18)

In [25]:
nulls = df_val.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,25553
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,25553
trip_distance,0
RatecodeID,25553
store_and_fwd_flag,25553
PULocationID,0
DOLocationID,0
payment_type,25553


In [26]:
df_val = df_val.dropna(how ='any')

In [27]:
nulls = df_val.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
store_and_fwd_flag,0
PULocationID,0
DOLocationID,0
payment_type,0


In [28]:
df_val['Day'] = pd.to_datetime(df_val['tpep_pickup_datetime']).dt.day

In [29]:
df_val['Hour_pickup'] = pd.to_datetime(df_val['tpep_pickup_datetime']).dt.hour
df_val

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
1413855,2.0,2021-03-25 14:27:14,2021-03-25 14:36:50,1.0,1.60,1.0,N,114,87,1.0,8.5,0.0,0.5,2.36,0.00,0.3,14.16,2.5,25,14
421074,1.0,2021-03-08 17:45:05,2021-03-08 18:24:11,1.0,13.00,1.0,N,161,14,1.0,38.5,3.5,0.5,14.65,6.12,0.3,63.57,2.5,8,17
935124,1.0,2021-03-17 15:23:13,2021-03-17 15:35:45,0.0,1.30,1.0,N,186,233,2.0,9.5,2.5,0.5,0.00,0.00,0.3,12.80,2.5,17,15
1704182,2.0,2021-03-30 15:53:17,2021-03-30 16:05:43,1.0,2.21,1.0,N,246,211,1.0,10.0,0.0,0.5,1.00,0.00,0.3,14.30,2.5,30,15
1164784,1.0,2021-03-21 07:40:56,2021-03-21 08:05:24,3.0,14.80,1.0,N,132,165,1.0,41.0,0.0,0.5,8.35,0.00,0.3,50.15,0.0,21,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542040,2.0,2021-03-10 19:02:43,2021-03-10 19:20:26,1.0,7.61,1.0,N,262,13,1.0,23.5,1.0,0.5,5.56,0.00,0.3,33.36,2.5,10,19
71889,2.0,2021-03-02 12:28:09,2021-03-02 12:34:46,1.0,1.36,1.0,N,234,100,1.0,6.5,0.0,0.5,1.96,0.00,0.3,11.76,2.5,2,12
1059358,2.0,2021-03-19 13:09:13,2021-03-19 13:27:06,2.0,3.87,1.0,N,237,249,1.0,15.5,0.0,0.5,3.76,0.00,0.3,22.56,2.5,19,13
188647,2.0,2021-03-04 12:39:06,2021-03-04 12:49:36,1.0,2.03,1.0,N,246,211,1.0,9.0,0.0,0.5,1.23,0.00,0.3,13.53,2.5,4,12


In [30]:
df1_val= df_val.drop(df_val[df_val.Day < 20 ].index)
df1_val

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
1413855,2.0,2021-03-25 14:27:14,2021-03-25 14:36:50,1.0,1.60,1.0,N,114,87,1.0,8.5,0.0,0.5,2.36,0.0,0.3,14.16,2.5,25,14
1704182,2.0,2021-03-30 15:53:17,2021-03-30 16:05:43,1.0,2.21,1.0,N,246,211,1.0,10.0,0.0,0.5,1.00,0.0,0.3,14.30,2.5,30,15
1164784,1.0,2021-03-21 07:40:56,2021-03-21 08:05:24,3.0,14.80,1.0,N,132,165,1.0,41.0,0.0,0.5,8.35,0.0,0.3,50.15,0.0,21,7
1365329,2.0,2021-03-24 17:41:44,2021-03-24 17:42:04,1.0,0.00,2.0,N,238,238,2.0,52.0,4.5,0.5,0.00,0.0,0.3,59.80,2.5,24,17
1171779,2.0,2021-03-21 12:00:20,2021-03-21 12:05:05,1.0,1.06,1.0,N,263,43,1.0,5.5,0.0,0.5,0.44,0.0,0.3,9.24,2.5,21,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130910,1.0,2021-03-20 15:01:36,2021-03-20 15:23:54,3.0,3.10,1.0,N,138,129,2.0,17.5,0.0,0.5,0.00,0.0,0.3,18.30,0.0,20,15
1296176,1.0,2021-03-23 16:18:53,2021-03-23 16:25:13,1.0,1.90,1.0,N,238,50,1.0,8.0,3.5,0.5,2.45,0.0,0.3,14.75,2.5,23,16
1684408,1.0,2021-03-30 10:59:34,2021-03-30 11:02:09,1.0,0.50,1.0,N,237,237,1.0,4.0,2.5,0.5,1.45,0.0,0.3,8.75,2.5,30,10
1768027,2.0,2021-03-31 15:39:48,2021-03-31 15:49:46,2.0,1.14,1.0,N,79,144,1.0,8.0,0.0,0.5,1.00,0.0,0.3,12.30,2.5,31,15


In [31]:
df1_val['Day'] = pd.to_datetime(df1_val['tpep_pickup_datetime']).dt.weekday

In [32]:
df1_val.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','tpep_pickup_datetime','tpep_dropoff_datetime']
         ,axis=1, inplace=True ) # delete two columns ,(axis=1:refers to column dimension)

In [70]:
duplicate = df1_val.duplicated()
print(duplicate.sum())
df1_val[duplicate]

5243


,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup
1322511,1.0,1.0,1.00,1.0,1.0,5.5,2.5,0.5,2.20,0.0,0.3,11.00,2.5,2,8
1675506,2.0,1.0,1.66,1.0,1.0,7.0,0.0,0.5,2.06,0.0,0.3,12.36,2.5,1,7
1784379,1.0,1.0,1.60,1.0,1.0,9.0,3.5,0.5,2.65,0.0,0.3,15.95,2.5,2,18
1641280,2.0,1.0,1.14,1.0,2.0,6.0,0.0,0.5,0.00,0.0,0.3,9.30,2.5,0,13
1361007,1.0,1.0,1.40,1.0,2.0,7.5,3.5,0.5,0.00,0.0,0.3,11.80,2.5,2,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163183,2.0,1.0,1.21,1.0,1.0,6.0,0.5,0.5,1.96,0.0,0.3,11.76,2.5,6,2
1345412,1.0,1.0,1.70,1.0,2.0,8.0,2.5,0.5,0.00,0.0,0.3,11.30,2.5,2,14
1253694,2.0,1.0,1.60,1.0,2.0,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0,20
1296176,1.0,1.0,1.90,1.0,1.0,8.0,3.5,0.5,2.45,0.0,0.3,14.75,2.5,1,16


In [71]:
df1_val.drop_duplicates(inplace=True)

In [72]:
duplicate = df1_val.duplicated()
print(duplicate.sum())
df1_val[duplicate]

0


,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup


In [33]:
df1_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139066 entries, 1413855 to 1408736
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               139066 non-null  float64
 1   passenger_count        139066 non-null  float64
 2   trip_distance          139066 non-null  float64
 3   RatecodeID             139066 non-null  float64
 4   payment_type           139066 non-null  float64
 5   fare_amount            139066 non-null  float64
 6   extra                  139066 non-null  float64
 7   mta_tax                139066 non-null  float64
 8   tip_amount             139066 non-null  float64
 9   tolls_amount           139066 non-null  float64
 10  improvement_surcharge  139066 non-null  float64
 11  total_amount           139066 non-null  float64
 12  congestion_surcharge   139066 non-null  float64
 13  Day                    139066 non-null  int64  
 14  Hour_pickup            139066

In [34]:
Day_hours = df1_val.groupby('Hour_pickup').Day.value_counts().sort_values(ascending=False)
Day_hours

Hour_pickup  Day
17           2      2200
18           2      2087
15           2      1928
16           2      1904
17           5      1884
                    ... 
2            3        16
3            3        15
             1        14
4            4        13
             3        12
Name: Day, Length: 168, dtype: int64

In [ ]:
#------- End clean val

In [55]:
df1_train['RatecodeID'] = df1_train.astype(str)
df1_train['RatecodeID']

1602218    1.0
1502881    2.0
1374167    2.0
1398315    2.0
1201170    2.0
          ... 
1118566    2.0
1455407    2.0
1245787    2.0
1409311    1.0
1629764    1.0
Name: RatecodeID, Length: 416891, dtype: object

In [56]:
df_dummies_train = pd.get_dummies(data=df1_train)
df_dummies_train.head() 

,VendorID,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day,Hour_pickup,RatecodeID_1.0,RatecodeID_2.0
1602218,1.0,3.0,1.10,1.0,6.0,2.5,0.5,1.00,0.0,0.3,10.30,2.5,6,15,1,0
1502881,2.0,1.0,2.19,2.0,13.5,1.0,0.5,0.00,0.0,0.3,17.80,2.5,4,19,0,1
1374167,2.0,1.0,2.38,1.0,9.5,1.0,0.5,2.76,0.0,0.3,16.56,2.5,2,19,0,1
1398315,2.0,1.0,0.86,1.0,5.0,0.0,0.5,1.66,0.0,0.3,9.96,2.5,3,11,0,1
1201170,2.0,2.0,2.88,1.0,12.0,0.5,0.5,3.95,0.0,0.3,19.75,2.5,6,22,0,1


In [57]:
X , y = df1_train[['trip_distance','RatecodeID','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','Hour_pickup','Day']], df1_train['fare_amount']

In [58]:
lr = linear_model.LinearRegression()
lr.fit(X,y)

LinearRegression()

In [59]:
lr.score(X,y)

0.7991290929127638

In [60]:
X_val , y_val = df1_val[['trip_distance','RatecodeID','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge']], df1_val['fare_amount']

In [61]:
lr = linear_model.LinearRegression()
lr.fit(X_val,y_val)

LinearRegression()

In [62]:
lr.score(X_val,y_val)

0.8146837367496121

In [ ]:
#------------------------- 

In [63]:
poly = PolynomialFeatures(degree = 4)
X_poly = poly.fit_transform(X)
 
poly.fit(X_poly, y)
lin2 = LinearRegression()
lin2.fit(X_poly, y)

LinearRegression()